### Create folder structure:

In [13]:
import os

# folder names
root_folder = 'classif_donorbased'
seeds = ['seed17', 'seed42', 'seed60', 'seed83']
anns = ['ann_2', 'ann_3', 'ann_4', 'ann_finest']
classifs = ['knn_classif', 'rf_classif']
result_folders = ['cms', 'norm_cms']

for seed in seeds:
    for ann in anns:
        for classif in classifs:
            for result_folder in result_folders:
                path = os.path.join(root_folder, seed, ann, classif, result_folder)
                os.makedirs(path, exist_ok = True)

### Functions to create templates:

In [14]:
def get_label_assignment(ann_value):
    # for 'None' entries, go back and get coarser cell type label
    if ann_value == "adata.obs.ann_level_3":
        return (
            "cell_type_labels = adata.obs['ann_level_3'].astype(str)\n"
            "cell_type_labels = cell_type_labels.where(cell_type_labels != 'None', adata.obs['ann_level_2'].astype(str))"
        )
    elif ann_value == "adata.obs.ann_level_4":
        return (
            "cell_type_labels = adata.obs['ann_level_4'].astype(str)\n"
            "cell_type_labels = cell_type_labels.where(cell_type_labels != 'None', adata.obs['ann_level_3'].astype(str))\n"
            "cell_type_labels = cell_type_labels.where(cell_type_labels != 'None', adata.obs['ann_level_2'].astype(str))"
        )
    else:
        # no need to handle 'None's for ann_2 and ann_finest
        return f"cell_type_labels = {ann_value}"

In [15]:
# KNN template
def generate_knn_script(prop_value, ann_value):
    label_assignment = get_label_assignment(ann_value)
    return f"""# KNN classification script

import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix
import seaborn as sns

print('successfully imported packages!', flush=True)

path = '/winhome/noraghenciules/hlca_data.h5ad'
adata = anndata.read_h5ad(path)

print('successfully read file!', flush=True)

embedding = adata.obsm['X_scanvi_emb']
sex_labels = adata.obs['sex']
individual_labels = adata.obs['donor_id']
{label_assignment}
classes = sorted(list(set(cell_type_labels)))

from helper_functions_final import final_train_clf_and_predict, final_evaluate_clf, plot_confusion, fixed_select_indices_by_proportion, check_missing_classes_in_folds

# ----------
# Run on data increasing female proportion:
    # (results are printed and plots are saved)

prop = {prop_value}

print(f"PROPORTION OF FEMALE CELLS: {{prop}}", flush=True)
print('Training and testing...', flush=True)
male_pred, male_true_labels, female_pred, female_true_labels = final_train_clf_and_predict(embedding, cell_type_labels, sex_labels, individual_labels, prop)
print('Evaluating...', flush=True)
male_metrics = final_evaluate_clf(male_pred, male_true_labels, classes, prop, 'male')
female_metrics = final_evaluate_clf(female_pred, female_true_labels, classes, prop, 'female')

# File path to save the dictionary
male_file_path = f"{{''.join(str(prop).split('.'))}}_male_metrics.pickle"
female_file_path = f"{{''.join(str(prop).split('.'))}}_female_metrics.pickle"

# Save the dictionary to a file
with open(male_file_path, 'wb') as file:
    pickle.dump(male_metrics, file)

with open(female_file_path, 'wb') as file:
    pickle.dump(female_metrics, file)
"""

# RF template
def generate_rf_script(prop_value, ann_value):
    label_assignment = get_label_assignment(ann_value)
    return f"""# RF classification script
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix
import seaborn as sns

print('successfully imported packages!', flush=True)

path = '/winhome/noraghenciules/hlca_data.h5ad'
adata = anndata.read_h5ad(path)

print('successfully read file!', flush=True)

counts = adata.X
sex_labels = adata.obs['sex']
individual_labels = adata.obs['donor_id']
{label_assignment}
classes = sorted(list(set(cell_type_labels)))

from helper_functions import train_clf_and_predict, evaluate_clf, plot_confusion, fixed_select_indices_by_proportion, check_missing_classes_in_folds

# ----------
# Run on data increasing female proportion:
    # (results are printed and plots are saved)

prop = {prop_value}

print(f"PROPORTION OF FEMALE CELLS: {{prop}}", flush=True)
print('Training and testing...', flush=True)
male_pred, male_true_labels, female_pred, female_true_labels = final_train_clf_and_predict(counts, cell_type_labels, sex_labels, individual_labels, prop, classifier = 'rf')
print('Evaluating...', flush=True)
male_metrics = evaluate_clf(male_pred, male_true_labels, classes, prop, 'male')
female_metrics = evaluate_clf(female_pred, female_true_labels, classes, prop, 'female')

# File path to save the dictionary
male_file_path = f"{{''.join(str(prop).split('.'))}}_male_metrics.pickle"
female_file_path = f"{{''.join(str(prop).split('.'))}}_female_metrics.pickle"

# Save the dictionary to a file
with open(male_file_path, 'wb') as file:
    pickle.dump(male_metrics, file)

with open(female_file_path, 'wb') as file:
    pickle.dump(female_metrics, file)

"""

In [16]:
# helper functions file
def generate_helper_file(seed_value):
    return f"""# Helper functions

import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import matplotlib.pyplot as plt
from scipy.sparse import vstack

from sklearn.model_selection import StratifiedKFold, train_test_split, GroupShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, precision_score
import seaborn as sns


# Train and evaluate functions:

def train_clf_and_predict(X, y, sex_labels, individual_labels, proportion_female, classifier='knn', k = 30):
    '''
    Parameters:
    X = expression matrix; matrix of shape n_obs x n_vars
    y = (cell type) labels; array/list of shape n_obs
    sex_labels = 'male' or 'female' label for each entry; array/list of shape n_obs
    individual_labels = donor id for each entry; array/list of shape n_obs
    proportion_female = desired proportion of female cells; float between 0 and 1
    classifier = 'knn' or 'rf'; default 'knn'
    k = number of neighbors if using knn; default 30
    ----------
    
    Returns:

    male_pred, female_pred = arrays of shape n_obs; contains the prediction on the male 
                                or female test set
    y_male_test, y_female_test = arrays of shape n_obs; contains the true labels of the 
                                male or female test set
    
    '''
    
    np.random.seed({seed_value})
    
    
    male_indices = np.where(sex_labels == 'male')[0]
    female_indices = np.where(sex_labels == 'female')[0]

    X_male = X[male_indices]
    y_male = y[male_indices]
    individual_labels_male = np.array(individual_labels)[male_indices]

    X_female = X[female_indices]
    y_female = y[female_indices]
    individual_labels_female = np.array(individual_labels)[female_indices]
    
    # GroupShuffleSplit to ensure no overlap of individuals
    gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state={seed_value})

    # Split female data
    for train_idx, test_idx in gss.split(X_female, y_female, groups=individual_labels_female):
        X_female_train, X_female_test = X_female[train_idx], X_female[test_idx]
        y_female_train, y_female_test = y_female[train_idx], y_female[test_idx]
        individual_labels_female_train = individual_labels_female[train_idx]
        individual_labels_female_test = individual_labels_female[test_idx]

    # Split male data with same test size ratio
    for train_idx, test_idx in gss.split(X_male, y_male, groups=individual_labels_male):
        X_male_train, X_male_test = X_male[train_idx], X_male[test_idx]
        y_male_train, y_male_test = y_male[train_idx], y_male[test_idx]
        individual_labels_male_train = individual_labels_male[train_idx]
        individual_labels_male_test = individual_labels_male[test_idx]

    # ensure equal sized test sets
    # first, shuffle:
    male_shuffle_indices = np.random.permutation(X_male_test.shape[0])
    X_male_test = X_male_test[male_shuffle_indices]
    y_male_test = y_male_test[male_shuffle_indices]
    female_shuffle_indices = np.random.permutation(X_female_test.shape[0])
    X_female_test = X_female_test[female_shuffle_indices]
    y_female_test = y_female_test[female_shuffle_indices]

    # then extract equal number of entries for both sexes:
    min_test_size = min(X_male_test.shape[0], X_female_test.shape[0])
    X_male_test = X_male_test[:min_test_size]
    y_male_test = y_male_test[:min_test_size]
    X_female_test = X_female_test[:min_test_size]
    y_female_test = y_female_test[:min_test_size]

    
    # merge training sets back together
    X_train = vstack([X_male_train, X_female_train])
    y_train = np.concatenate([y_male_train, y_female_train])
    sex_labels_train = ['male'] * X_male_train.shape[0] + ['female'] * X_female_train.shape[0]
    ind_train = np.concatenate([individual_labels_male_train, individual_labels_female_train])

    # Select female cells based on proportion_female
    selected_indices = fixed_select_indices_by_proportion(sex_labels_train, proportion_female)
    X_selected = X_train.tocsr()[selected_indices]
    y_selected = y_train[selected_indices]

    # Initialize classifier
    if classifier == 'knn':
        clf = KNeighborsClassifier(n_neighbors=k)
    elif classifier == 'rf':
        clf = RandomForestClassifier(n_jobs=-1)
        

    print('initialized classif!', flush = True)
    # Train
    clf.fit(X_selected, y_selected)
       
    print('done training!', flush = True)
    
    # Predict
    male_pred = clf.predict(X_male_test)
    female_pred = clf.predict(X_female_test)
     
    return male_pred, y_male_test, female_pred, y_female_test


def evaluate_clf(predictions, true_labels, classes, prop, sex):
    '''
    This is meant to be run separately on the male and female results of the train function.
    ---------
    
    Parameters:

    predictions = predictions on a test set
    true_labels = true labels on that test set
    classes = sorted list of classes
    prop = proportion of female cells
    sex = 'male' or 'female'; dneotes what test set we are working with
        # prop and sex are only used for naming the confusion matrix plots
    
    ----------
    
    Returns:

    accuracy = accuracy score
    f1_per_class = array of shape n_classes; each entry is the f1 score of that class
    median_f1 = median of the f1_per_class array
    precision_per_class = array of shape n_classes; each entry is the precision score of that class
    median_precision = median of the precision_per_class array
    cm = confusion matrix
    cm_normalized = normalized confusion matrix
        (the function saves plots of each confusion matrix)
    
    '''
    n_classes = len(classes)

    # Accuracy scores
    accuracy = accuracy_score(true_labels, predictions)

    # F1 per class
    f1_per_class = f1_score(true_labels, predictions, average=None)

    # Median F1
    median_f1 = np.median(f1_per_class)

    # Precision per class
    precision_per_class = precision_score(true_labels, predictions, average=None)

    # Median precision
    median_precision = np.median(precision_per_class)

    # Confusion matrix
    cm = confusion_matrix(true_labels, predictions, labels=classes)
    # Normalized confusion matrix:
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    
    # Create dictionary
    metrics = {{
        'accuracy': accuracy,
        'f1_scores': f1_per_class,
        'median_f1': median_f1,
        'precision_scores': precision_per_class,
        'median_precision': median_precision,
        'aggregated_confusion_matrix': cm,
        'normalized_aggregated_confusion_matrix': cm_normalized
    }}
        
        
    plot_confusion(cm, classes, f'Prop {{prop}}, {{sex}} test set Confusion Matrix', False)
    plot_confusion(cm_normalized, classes, f'Prop {{prop}}, {{sex}} test set Normalized Confusion Matrix', True)
    
    
    return metrics



# Helper functions:

def plot_confusion(confusion_matrix, classes, title, normalize = False):
    '''
    Plot and save the current confusion matrix.
    Make sure to pass a title and the normalize parameter (if the matrix is normalized).
    '''
    
    plt.clf()
    plt.figure(figsize=(10, 8))
    if normalize:
        sns.heatmap(confusion_matrix, annot=True, fmt=".3f", cmap="Blues", xticklabels=classes, yticklabels=classes)
    else:
        sns.heatmap(confusion_matrix, annot=True, fmt="g", cmap="Blues", xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(title)
    if normalize:
        plt.savefig(f'norm_cms/{{title}}.png', bbox_inches='tight')
    else:
        plt.savefig(f'cms/{{title}}.png', bbox_inches='tight')
    plt.close()


def fixed_select_indices_by_proportion(sex_labels, proportion_female):
    np.random.seed({seed_value})
    sex_labels_series = pd.Series( (el for el in sex_labels) )
    
    female_indices = np.where(sex_labels_series == 'female')[0]
    male_indices = np.where(sex_labels_series == 'male')[0]
    
    fixed_size = min(len(female_indices), len(male_indices))
    
    np.random.shuffle(female_indices)
    np.random.shuffle(male_indices)

    num_female_cells = int(fixed_size * proportion_female)
    num_male_cells = fixed_size - num_female_cells
        # total will always be fixed_size
        # this works for cases with prop 0% or 100% --> no need to handle them separately
    
    # adjust in case of rounding errors
    num_female_cells = min(num_female_cells, len(female_indices))
    num_male_cells = min(num_male_cells, len(male_indices))

    selected_female_indices = female_indices[:num_female_cells]
    selected_male_indices = male_indices[:num_male_cells]

    return np.concatenate([selected_female_indices, selected_male_indices])




def check_missing_classes_in_folds(predictions, true_labels, classes):
    '''
    Checks if there are folds that miss predictions or true labels for a class.
    The latter can happen if there are fewer samples of a certain class than folds.
    '''
    
    missing_info = [] 
    
    for fold_index, (y_pred, y_true) in enumerate(zip(predictions, true_labels)):
        unique_pred_classes = set(np.unique(y_pred))
        unique_true_classes = set(np.unique(y_true))
        all_classes_set = set(classes)

        missing_in_pred = all_classes_set - unique_pred_classes
        missing_in_true = all_classes_set - unique_true_classes

        if missing_in_pred or missing_in_true:
            missing_info.append({{
                'fold': fold_index,
                'missing_in_predictions': sorted(list(missing_in_pred)), 
                'missing_in_true_labels': sorted(list(missing_in_true))
            }})

    if missing_info:
        for info in missing_info:
            print(f"Fold {{info['fold']}} is missing predictions for classes: {{info['missing_in_predictions']}}")
            print(f"Fold {{info['fold']}} is missing true labels for classes: {{info['missing_in_true_labels']}}")
    else:
        print("All folds have predictions and true labels for all classes.")

"""


In [17]:
# SLURM scripts
def generate_knn_slurm_script(py_filename):
    return f"""#!/bin/sh
#SBATCH --partition=general   # Request partition. Default is 'general' 
#SBATCH --qos=short           # Request Quality of Service. Default is 'short' (maximum run time: 4 hours)
#SBATCH --time=0:40:00        # Request run time (wall-clock). Default is 1 minute
#SBATCH --ntasks=1            # Request number of parallel tasks per job. Default is 1
#SBATCH --cpus-per-task=2     # Request number of CPUs (threads) per task. Default is 1 (note: CPUs are always allocated to jobs per 2).
#SBATCH --mem=20GB             # Request memory (MB) per node. Default is 1024MB (1GB). For multiple tasks, specify --mem-per-cpu instead
#SBATCH --mail-type=FAIL       # Set mail type to 'END' to receive a mail when the job finishes. 
#SBATCH --output=slurm_%j.out # Set name of output log. %j is the Slurm jobId
#SBATCH --error=slurm_%j.err  # Set name of error log. %j is the Slurm jobId

# which python 1>&2  # Write path to Python binary to standard error
# python --version   # Write Python version to standard error

srun python3 {py_filename}
"""

# rf needs more memory + time
def generate_rf_slurm_script(py_filename):
    return f"""#!/bin/sh
#SBATCH --partition=general   # Request partition. Default is 'general' 
#SBATCH --qos=short           # Request Quality of Service. Default is 'short' (maximum run time: 4 hours)
#SBATCH --time=2:00:00        # Request run time (wall-clock). Default is 1 minute
#SBATCH --ntasks=1            # Request number of parallel tasks per job. Default is 1
#SBATCH --cpus-per-task=2     # Request number of CPUs (threads) per task. Default is 1 (note: CPUs are always allocated to jobs per 2).
#SBATCH --mem=80GB             # Request memory (MB) per node. Default is 1024MB (1GB). For multiple tasks, specify --mem-per-cpu instead
#SBATCH --mail-type=FAIL       # Set mail type to 'END' to receive a mail when the job finishes. 
#SBATCH --output=slurm_%j.out # Set name of output log. %j is the Slurm jobId
#SBATCH --error=slurm_%j.err  # Set name of error log. %j is the Slurm jobId

# which python 1>&2  # Write path to Python binary to standard error
# python --version   # Write Python version to standard error

srun python3 {py_filename}
"""

### Write classification files:

In [18]:
props = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

# dictionaries for notations:
ann_to_value = {
    'ann_2': "adata.obs.ann_level_2",
    'ann_3': "adata.obs.ann_level_3",
    'ann_4': "adata.obs.ann_level_4",
    'ann_finest': "adata.obs['ann_finest_level'].astype(str)"
}

prop_to_str = {
    0.0: "0", 0.1: "01", 0.2: "02", 0.3: "03", 0.4: "04", 0.5: "05",
    0.6: "06", 0.7: "07", 0.8: "08", 0.9: "09", 1.0: "1"
}

seed_to_value = {
    'seed17': 17,
    'seed42' : 42,
    'seed60' : 60,
    'seed83' : 83
}

# loop through folder structure
for seed in seeds:
    for ann in anns:
        for classif in classifs:
            # helper file
            helper_file_path = os.path.join(root_folder, seed, ann, classif, 'helper_functions.py')
            seed_value = seed_to_value[seed]
            with open(helper_file_path, 'w') as f:
                    f.write(generate_helper_file(seed_value))
            
            # classification files
            for prop in props:
                prop_str = prop_to_str[prop]
                filename = f"{classif.split('_')[0]}_{prop_str}.py"
                file_path = os.path.join(root_folder, seed, ann, classif, filename)

                sh_filename = f"{classif.split('_')[0]}_{prop_str}.sh"
                sh_path = os.path.join(root_folder, seed, ann, classif, sh_filename)

                
                ann_value = ann_to_value[ann]
                # write files
                if classif == 'knn_classif':
                    ## write SLURM file
                    with open(sh_path, 'w', newline='\n') as f_sh:
                        f_sh.write(generate_knn_slurm_script(filename))
                    
                    ## write .py file
                    with open(file_path, 'w') as f:
                        f.write(generate_knn_script(prop, ann_value))

                elif classif == 'rf_classif':
                    ## write SLURM file
                    with open(sh_path, 'w', newline='\n') as f_sh:
                        f_sh.write(generate_rf_slurm_script(filename))
                    
                    ## write .py file
                    with open(file_path, 'w') as f:
                        f.write(generate_rf_script(prop, ann_value))